### Preview
Preprocess step is conducted after you finish the step of tweet extraction. 

In [2]:
import pandas as pd
import re
import nltk
import string

#### Showing Dataframe
If you're wondering why the file title is not the same with the output I yield in tweet extraction step, it's because I have appended all files into one file named 'Raw Data_formal_initiation.xlsx'. If you want to get the same file like mine, you could repeat the same step and make a little change in timeframe to get additional tweets from January 2016 & February 2016.

In [3]:
df = pd.read_excel(r'Raw Data_formal_initiation.xlsx')
print(df)

       Unnamed: 0  Unnamed: 0.1            Datetime      Tweet Id  \
0               0             0 2015-07-30 23:58:35  6.269048e+17   
1               2             2 2015-07-30 23:58:26  6.269047e+17   
2               3             3 2015-07-30 23:58:13  6.269047e+17   
3               4             4 2015-07-30 23:53:06  6.269034e+17   
4               5             5 2015-07-30 23:49:28  6.269025e+17   
...           ...           ...                 ...           ...   
54648        5468           914 2016-02-01 03:56:43  6.940064e+17   
54649        5469           915 2016-02-01 03:49:21  6.940046e+17   
54650        5470           916 2016-02-01 03:40:31  6.940024e+17   
54651        5471           917 2016-02-01 02:12:25  6.939802e+17   
54652        5472           918 2016-02-01 00:00:35  6.939470e+17   

                                                    Text         Username  \
0      Avanza Disikat Truk di Gunungpati, 2 Orang Tew...  tamhaRHidayanto   
1      #BeritaSMG

#### Deleting duplicated data
I used the indicator of 'Text' and 'Tweet Id' to erase the duplicated data

In [4]:
df.drop_duplicates(subset ="Text", keep = 'first', inplace = True)
df.drop_duplicates(subset ="Tweet Id", keep = 'first', inplace = True)
df

,Unnamed: 0,Unnamed: 0.1,Datetime,Tweet Id,Text,Username,Text_formal
0,0,0,2015-07-30 23:58:35,6.269048e+17,"Avanza Disikat Truk di Gunungpati, 2 Orang Tew...",tamhaRHidayanto,"Avanza Disikat Truk di Gunungpati, 2 Orang Tew..."
1,2,2,2015-07-30 23:58:26,6.269047e+17,"#BeritaSMG Avanza Disikat Truk di Gunungpati, ...",jatengpedia,"#BeritaSMG Avanza Disikat Truk di Gunungpati, ..."
2,3,3,2015-07-30 23:58:13,6.269047e+17,"Nyuci baju to?kok disikat?""@endah_rahmaa: Avan...",pi_ping1,"Nyuci baju to?kok disikat?""@endah_rahmaa: Avan..."
3,4,4,2015-07-30 23:53:06,6.269034e+17,Terimakasih transaksinya pak Fahri headlamp st...,imskemayoran,Terimakasih transaksinya pak Fahri headlamp st...
4,5,5,2015-07-30 23:49:28,6.269025e+17,Video: Intip bocoran utuh wujud Toyota Avanza ...,kabarlintasme,Video: Intip bocoran utuh wujud Toyota Avanza ...
...,...,...,...,...,...,...,...
54648,5468,914,2016-02-01 03:56:43,6.940064e+17,Toyota Fortuner Generasi Baru Lebih Kedap dan ...,Mobil123ID,Toyota Fortuner Generasi Baru Lebih Kedap dan ...
54649,5469,915,2016-02-01 03:49:21,6.940046e+17,Pagi2 sudah disuguhi teman2 pecinta pahlawan I...,renny_amanda,Pagi2 sudah disuguhi teman-teman pecinta pahla...
54650,5470,916,2016-02-01 03:40:31,6.940024e+17,Toyota Fortuner Diesel Lebih Menguntungkan \n\...,Mobil123ID,Toyota Fortuner Diesel Lebih Menguntungkan @To...
54651,5471,917,2016-02-01 02:12:25,6.939802e+17,Toyota Caplok Produsen Mobil Tertua di Jepang ...,Mobil123ID,Toyota Caplok Produsen Mobil Tertua di Jepang ...


In [5]:
#If there is null data (NaN) in "Text" column, change it into empty string('')
df['Text'] = df['Text'].fillna('').apply(str)

#remove link in the text
#e.g: the text is "halo https:\\www.example.com".
#The output would be only "halo"
df['Text'] = df['Text'].str.replace('http\S+|www.\S+', '', case=False)

#Removed duplicate data
df.drop_duplicates(subset ="Text", keep = 'first', inplace = True)

<ipython-input-5-f8e263370ce5>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Text'] = df['Text'].str.replace('http\S+|www.\S+', '', case=False)


#### Formatting Slang into Formal
Tweets are mostly full of slangs. Sentiment analysis won't be able to analyze the text since VADER is only working for formal english dictionary. Since these tweets are still in Indonesian and full of slangs, we're going to replace the informal words with their formal pairs. The slang dictionary used here is from "https://raw.githubusercontent.com/nasalsabila/kamus-alay/master/colloquial-indonesian-lexicon.csv"

In [6]:
slang = pd.read_csv("https://raw.githubusercontent.com/nasalsabila/kamus-alay/master/colloquial-indonesian-lexicon.csv")
slang

,slang,formal,In-dictionary,context,category1,category2,category3
0,woww,wow,1,wow,elongasi,0,0
1,aminn,amin,1,Selamat ulang tahun kakak tulus semoga panjang...,elongasi,0,0
2,met,selamat,1,Met hari netaas kak!? Wish you all the best @t...,abreviasi,0,0
3,netaas,menetas,1,Met hari netaas kak!? Wish you all the best @t...,afiksasi,elongasi,0
4,keberpa,keberapa,0,Birthday yg keberpa kak?,abreviasi,0,0
...,...,...,...,...,...,...,...
15001,gataunya,enggak taunya,0,Ini kaya nenek2 ya beb gataunya agnezz @yugime...,akronim,0,0
15002,gtau,enggak tau,0,Stidaknya mrka may berkarya Dan berusaha yg tr...,akronim,abreviasi,0
15003,gatau,enggak tau,0,Ih gatau malu,akronim,0,0
15004,fans2,fan-fan,0,Jkt48 adalah tempat di mana sesama fans saling...,reduplikasi,naturalisasi,0


In [7]:
# convert data slang ke format dictionary
slangDict = dict(zip(slang['slang'], slang['formal']))

#membuat sebuah fungsi, untuk mengganti slang ke bahasa baku dari text
def replace_slang(text):
    text = text.lower()
    informalList = text.split()
    formalList = []

    for word in informalList:
        if word in slangDict:
            word = word.replace(word, slangDict[word])

        formalList.append(word)

    return " ".join(formalList)

df ['Text_formal']= df ['Text'].apply(replace_slang)

#### Erasing tweets from unwanted accounts
The raw data is still full of tweets from unwanted accounts such as buzzers, ads, Avanza car sellers. The purpose of this process is to get the tweets specifically from real Avanza car users. Hence, the sentiment analysis result would be more objective since it comes from the real users.

Unfortunately, I still pick unwanted accounts manually and put it in a file named 'akun_oa.xlsx'. You could suggest on how to get unwanted accounts faster to better this project here.

In [8]:
#menyimpan isi dari file akunooa.xlsx ke variable dataakun agar bisa diproses
dataakun = pd.read_excel(r'akun_oa.xlsx')
#menampilkan isi
print(dataakun)

                 Akun
0     tamhaRHidayanto
1     WelcomeSemarang
2         jatengpedia
3        imskemayoran
4       kabarlintasme
...               ...
1612        IkaMaliqa
1613    D4f_DreamTeam
1614   ekanurrohmah99
1615      InfoBali_ID
1616         Qianz_94

[1617 rows x 1 columns]


In [9]:
search_values= dataakun['Akun'] 

#filter to get the tweets from unwanted accounts
df[df.Username.str.match('|'.join(search_values), na=False)]

,Unnamed: 0,Unnamed: 0.1,Datetime,Tweet Id,Text,Username,Text_formal
0,0,0,2015-07-30 23:58:35,6.269048e+17,"Avanza Disikat Truk di Gunungpati, 2 Orang Tew...",tamhaRHidayanto,"avanza disikat truk di gunungpati, 2 orang tew..."
1,2,2,2015-07-30 23:58:26,6.269047e+17,"#BeritaSMG Avanza Disikat Truk di Gunungpati, ...",jatengpedia,"#beritasmg avanza disikat truk di gunungpati, ..."
2,3,3,2015-07-30 23:58:13,6.269047e+17,"Nyuci baju to?kok disikat?""@endah_rahmaa: Avan...",pi_ping1,"nyuci baju to?kok disikat?""@endah_rahmaa: avan..."
3,4,4,2015-07-30 23:53:06,6.269034e+17,Terimakasih transaksinya pak Fahri headlamp st...,imskemayoran,terimakasih transaksinya pak fahri headlamp st...
4,5,5,2015-07-30 23:49:28,6.269025e+17,Video: Intip bocoran utuh wujud Toyota Avanza ...,kabarlintasme,video: intip bocoran utuh wujud toyota avanza ...
...,...,...,...,...,...,...,...
54645,5465,911,2016-02-01 05:30:05,6.940299e+17,Ternyata faktor-faktor ini bisa mempengaruhi s...,ToyotaID,ternyata faktor-faktor ini bisa mempengaruhi s...
54646,5466,912,2016-02-01 05:28:38,6.940296e+17,@inilahdotcom #terpopuler #news Toyota Indones...,anandagadis1,@inilahdotcom #terpopuler #news toyota indones...
54648,5468,914,2016-02-01 03:56:43,6.940064e+17,Toyota Fortuner Generasi Baru Lebih Kedap dan ...,Mobil123ID,toyota fortuner generasi baru lebih kedap dan ...
54650,5470,916,2016-02-01 03:40:31,6.940024e+17,Toyota Fortuner Diesel Lebih Menguntungkan \n\...,Mobil123ID,toyota fortuner diesel lebih menguntungkan @to...


In [12]:
#filter to get the data from NON-unwanted accounts.
removeoa = df.drop(df[df['Username'].str.match('|'.join(search_values) , na= False)].index)
#menampilkan isi data
print(removeoa)

       Unnamed: 0  Unnamed: 0.1            Datetime      Tweet Id  \
41             83            83 2015-07-30 19:18:48  6.268343e+17   
93            283           283 2015-07-30 16:40:43  6.267946e+17   
95            286           286 2015-07-30 16:37:31  6.267938e+17   
104           301           301 2015-07-30 16:17:15  6.267887e+17   
116           313           313 2015-07-30 15:33:07  6.267775e+17   
...           ...           ...                 ...           ...   
54635        5455           901 2016-02-01 13:40:05  6.941532e+17   
54642        5462           908 2016-02-01 07:23:03  6.940584e+17   
54647        5467           913 2016-02-01 05:00:28  6.940225e+17   
54649        5469           915 2016-02-01 03:49:21  6.940046e+17   
54652        5472           918 2016-02-01 00:00:35  6.939470e+17   

                                                    Text        Username  \
41     Avanza bakal tampil dengan mesin baru yang leb...  BinandaSoraten   
93     Pkoknya. Nmp

'removeoa' is cleaned data coming from real users. We could use this data for the next preprocessing step.

#### Removing number 
Numbers can't be detected in sentiment analysis so we could remove numbers here.

In [13]:
#REMOVE NUMBER

removeoa['text_cleaning'] = removeoa['Text_formal'].str.replace('\d+', '')

<ipython-input-13-74895dd084ff>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  removeoa['text_cleaning'] = removeoa['Text_formal'].str.replace('\d+', '')


#### Remove Usernames, Hashtag, Retweet, Hyperlink

In [14]:
#membuat sebuah fungsi, untuk menghilangkan usernames,hashtags,RT,hyperlink dari text
def cleanTxt(text):
    text = re.sub('@[^\s]+', '', text) # remove usernames
    text = re.sub("#[A-Za-z0-9_]+","", str (text)) #remove hashtags
    text = re.sub(r'RT[\s]+', '',str (text)) #remove RT
    text = re.sub(r'https?:\/\/\S+', '', str (text))  #remove hyperlink 
   
    return text
#menggunakan fungsi tadi untuk membersihkan Text pada data duplicateremoved.xlsx
removeoa['text_cleaning']= removeoa['text_cleaning'].apply(cleanTxt)

removeoa.drop_duplicates(subset ="text_cleaning", keep = 'first', inplace = True)

#### Remove Punctuation

In [15]:
import string

#membuat sebuah fungsi, untuk menghilangkan punctuation dari text
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    return text

#menggunakan fungsi tadi untuk membersihkan Text pada data duplicateremoved.xlsx
removeoa['Text_punct']= removeoa['text_cleaning'].apply(remove_punct)

removeoa.drop_duplicates(subset ="Text_punct", keep = 'first', inplace = True)

#### Make text in lowercase

In [16]:
#lowercase

#membuat sebuah fungsi, untuk menghilangkan punctuation dari text
def lowercase(text):
    text  = text.lower()
    return text

removeoa['Text_lower']= removeoa['Text_punct'].apply(lowercase)

removeoa.drop_duplicates(subset ="Text_lower", keep = 'first', inplace = True)

#### Remove Emoticons

In [17]:
#remove emoticons

EMOTICONS = {
    u":‑\)":"Happy face or smiley",
    u":\)":"Happy face or smiley",
    u":-\]":"Happy face or smiley",
    u":\]":"Happy face or smiley",
    u":-3":"Happy face smiley",
    u":3":"Happy face smiley",
    u":->":"Happy face smiley",
    u":>":"Happy face smiley",
    u"8-\)":"Happy face smiley",
    u":o\)":"Happy face smiley",
    u":-\}":"Happy face smiley",
    u":\}":"Happy face smiley",
    u":-\)":"Happy face smiley",
    u":c\)":"Happy face smiley",
    u":\^\)":"Happy face smiley",
    u"=\]":"Happy face smiley",
    u"=\)":"Happy face smiley",
    u":‑D":"Laughing, big grin or laugh with glasses",
    u":D":"Laughing, big grin or laugh with glasses",
    u"8‑D":"Laughing, big grin or laugh with glasses",
    u"8D":"Laughing, big grin or laugh with glasses",
    u"X‑D":"Laughing, big grin or laugh with glasses",
    u"XD":"Laughing, big grin or laugh with glasses",
    u"=D":"Laughing, big grin or laugh with glasses",
    u"=3":"Laughing, big grin or laugh with glasses",
    u"B\^D":"Laughing, big grin or laugh with glasses",
    u":-\)\)":"Very happy",
    u":‑\(":"Frown, sad, andry or pouting",
    u":-\(":"Frown, sad, andry or pouting",
    u":\(":"Frown, sad, andry or pouting",
    u":‑c":"Frown, sad, andry or pouting",
    u":c":"Frown, sad, andry or pouting",
    u":‑<":"Frown, sad, andry or pouting",
    u":<":"Frown, sad, andry or pouting",
    u":‑\[":"Frown, sad, andry or pouting",
    u":\[":"Frown, sad, andry or pouting",
    u":-\|\|":"Frown, sad, andry or pouting",
    u">:\[":"Frown, sad, andry or pouting",
    u":\{":"Frown, sad, andry or pouting",
    u":@":"Frown, sad, andry or pouting",
    u">:\(":"Frown, sad, andry or pouting",
    u":'‑\(":"Crying",
    u":'\(":"Crying",
    u":'‑\)":"Tears of happiness",
    u":'\)":"Tears of happiness",
    u"D‑':":"Horror",
    u"D:<":"Disgust",
    u"D:":"Sadness",
    u"D8":"Great dismay",
    u"D;":"Great dismay",
    u"D=":"Great dismay",
    u"DX":"Great dismay",
    u":‑O":"Surprise",
    u":O":"Surprise",
    u":‑o":"Surprise",
    u":o":"Surprise",
    u":-0":"Shock",
    u"8‑0":"Yawn",
    u">:O":"Yawn",
    u":-\*":"Kiss",
    u":\*":"Kiss",
    u":X":"Kiss",
    u";‑\)":"Wink or smirk",
    u";\)":"Wink or smirk",
    u"\*-\)":"Wink or smirk",
    u"\*\)":"Wink or smirk",
    u";‑\]":"Wink or smirk",
    u";\]":"Wink or smirk",
    u";\^\)":"Wink or smirk",
    u":‑,":"Wink or smirk",
    u";D":"Wink or smirk",
    u":‑P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"X‑P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"XP":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":‑Þ":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":Þ":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":b":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"d:":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"=p":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u">:P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":‑/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":-[.]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u">:[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u">:/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":L":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=L":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":S":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":‑\|":"Straight face",
    u":\|":"Straight face",
    u":$":"Embarrassed or blushing",
    u":‑x":"Sealed lips or wearing braces or tongue-tied",
    u":x":"Sealed lips or wearing braces or tongue-tied",
    u":‑#":"Sealed lips or wearing braces or tongue-tied",
    u":#":"Sealed lips or wearing braces or tongue-tied",
    u":‑&":"Sealed lips or wearing braces or tongue-tied",
    u":&":"Sealed lips or wearing braces or tongue-tied",
    u"O:‑\)":"Angel, saint or innocent",
    u"O:\)":"Angel, saint or innocent",
    u"0:‑3":"Angel, saint or innocent",
    u"0:3":"Angel, saint or innocent",
    u"0:‑\)":"Angel, saint or innocent",
    u"0:\)":"Angel, saint or innocent",
    u":‑b":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"0;\^\)":"Angel, saint or innocent",
    u">:‑\)":"Evil or devilish",
    u">:\)":"Evil or devilish",
    u"\}:‑\)":"Evil or devilish",
    u"\}:\)":"Evil or devilish",
    u"3:‑\)":"Evil or devilish",
    u"3:\)":"Evil or devilish",
    u">;\)":"Evil or devilish",
    u"\|;‑\)":"Cool",
    u"\|‑O":"Bored",
    u":‑J":"Tongue-in-cheek",
    u"#‑\)":"Party all night",
    u"%‑\)":"Drunk or confused",
    u"%\)":"Drunk or confused",
    u":-###..":"Being sick",
    u":###..":"Being sick",
    u"<:‑\|":"Dump",
    u"\(>_<\)":"Troubled",
    u"\(>_<\)>":"Troubled",
    u"\(';'\)":"Baby",
    u"\(\^\^>``":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(\^_\^;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(-_-;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(~_~;\) \(・\.・;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(-_-\)zzz":"Sleeping",
    u"\(\^_-\)":"Wink",
    u"\(\(\+_\+\)\)":"Confused",
    u"\(\+o\+\)":"Confused",
    u"\(o\|o\)":"Ultraman",
    u"\^_\^":"Joyful",
    u"\(\^_\^\)/":"Joyful",
    u"\(\^O\^\)／":"Joyful",
    u"\(\^o\^\)／":"Joyful",
    u"\(__\)":"Kowtow as a sign of respect, or dogeza for apology",
    u"_\(\._\.\)_":"Kowtow as a sign of respect, or dogeza for apology",
    u"<\(_ _\)>":"Kowtow as a sign of respect, or dogeza for apology",
    u"<m\(__\)m>":"Kowtow as a sign of respect, or dogeza for apology",
    u"m\(__\)m":"Kowtow as a sign of respect, or dogeza for apology",
    u"m\(_ _\)m":"Kowtow as a sign of respect, or dogeza for apology",
    u"\('_'\)":"Sad or Crying",
    u"\(/_;\)":"Sad or Crying",
    u"\(T_T\) \(;_;\)":"Sad or Crying",
    u"\(;_;":"Sad of Crying",
    u"\(;_:\)":"Sad or Crying",
    u"\(;O;\)":"Sad or Crying",
    u"\(:_;\)":"Sad or Crying",
    u"\(ToT\)":"Sad or Crying",
    u";_;":"Sad or Crying",
    u";-;":"Sad or Crying",
    u";n;":"Sad or Crying",
    u";;":"Sad or Crying",
    u"Q\.Q":"Sad or Crying",
    u"T\.T":"Sad or Crying",
    u"QQ":"Sad or Crying",
    u"Q_Q":"Sad or Crying",
    u"\(-\.-\)":"Shame",
    u"\(-_-\)":"Shame",
    u"\(一一\)":"Shame",
    u"\(；一_一\)":"Shame",
    u"\(=_=\)":"Tired",
    u"\(=\^\·\^=\)":"cat",
    u"\(=\^\·\·\^=\)":"cat",
    u"=_\^=	":"cat",
    u"\(\.\.\)":"Looking down",
    u"\(\._\.\)":"Looking down",
    u"\^m\^":"Giggling with hand covering mouth",
    u"\(\・\・?":"Confusion",
    u"\(?_?\)":"Confusion",
    u">\^_\^<":"Normal Laugh",
    u"<\^!\^>":"Normal Laugh",
    u"\^/\^":"Normal Laugh",
    u"\（\*\^_\^\*）" :"Normal Laugh",
    u"\(\^<\^\) \(\^\.\^\)":"Normal Laugh",
    u"\(^\^\)":"Normal Laugh",
    u"\(\^\.\^\)":"Normal Laugh",
    u"\(\^_\^\.\)":"Normal Laugh",
    u"\(\^_\^\)":"Normal Laugh",
    u"\(\^\^\)":"Normal Laugh",
    u"\(\^J\^\)":"Normal Laugh",
    u"\(\*\^\.\^\*\)":"Normal Laugh",
    u"\(\^—\^\）":"Normal Laugh",
    u"\(#\^\.\^#\)":"Normal Laugh",
    u"\（\^—\^\）":"Waving",
    u"\(;_;\)/~~~":"Waving",
    u"\(\^\.\^\)/~~~":"Waving",
    u"\(-_-\)/~~~ \($\·\·\)/~~~":"Waving",
    u"\(T_T\)/~~~":"Waving",
    u"\(ToT\)/~~~":"Waving",
    u"\(\*\^0\^\*\)":"Excited",
    u"\(\*_\*\)":"Amazed",
    u"\(\*_\*;":"Amazed",
    u"\(\+_\+\) \(@_@\)":"Amazed",
    u"\(\*\^\^\)v":"Laughing,Cheerful",
    u"\(\^_\^\)v":"Laughing,Cheerful",
    u"\(\(d[-_-]b\)\)":"Headphones,Listening to music",
    u'\(-"-\)':"Worried",
    u"\(ーー;\)":"Worried",
    u"\(\^0_0\^\)":"Eyeglasses",
    u"\(\＾ｖ\＾\)":"Happy",
    u"\(\＾ｕ\＾\)":"Happy",
    u"\(\^\)o\(\^\)":"Happy",
    u"\(\^O\^\)":"Happy",
    u"\(\^o\^\)":"Happy",
    u"\)\^o\^\(":"Happy",
    u":O o_O":"Surprised",
    u"o_0":"Surprised",
    u"o\.O":"Surpised",
    u"\(o\.o\)":"Surprised",
    u"oO":"Surprised",
    u"\(\*￣m￣\)":"Dissatisfied",
    u"\(‘A`\)":"Snubbed or Deflated"
}

def remove_emoticons(text):
    emoticon_pattern = re.compile(u'(' + u'|'.join(k for k in EMOTICONS) + u')')
    return emoticon_pattern.sub(r'', text)

removeoa["text_wo_emoticons"] = removeoa["Text_lower"].apply(remove_emoticons)

removeoa.drop_duplicates(subset ="text_wo_emoticons", keep = 'first', inplace = True)

#### Remove Emoji

In [18]:
#REMOVE EMOJI

def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

removeoa["text_wo_emoji"] = removeoa["text_wo_emoticons"].apply(remove_emoji)

removeoa.drop_duplicates(subset ="text_wo_emoji", keep = 'first', inplace = True)

#### Remove Stopwords
Stopwords are the words which are generally filtered out before processing a natural language. These are actually the most common words in any language (like articles, prepositions, pronouns, conjunctions, etc) and does not add much information to the text. Therefore, we could delete them in preprocessing step.

In [19]:
#LIST STOPWORDS INDONESIA

from nltk.corpus import stopwords
", ".join(stopwords.words('indonesian'))

'ada, adalah, adanya, adapun, agak, agaknya, agar, akan, akankah, akhir, akhiri, akhirnya, aku, akulah, amat, amatlah, anda, andalah, antar, antara, antaranya, apa, apaan, apabila, apakah, apalagi, apatah, artinya, asal, asalkan, atas, atau, ataukah, ataupun, awal, awalnya, bagai, bagaikan, bagaimana, bagaimanakah, bagaimanapun, bagi, bagian, bahkan, bahwa, bahwasanya, baik, bakal, bakalan, balik, banyak, bapak, baru, bawah, beberapa, begini, beginian, beginikah, beginilah, begitu, begitukah, begitulah, begitupun, bekerja, belakang, belakangan, belum, belumlah, benar, benarkah, benarlah, berada, berakhir, berakhirlah, berakhirnya, berapa, berapakah, berapalah, berapapun, berarti, berawal, berbagai, berdatangan, beri, berikan, berikut, berikutnya, berjumlah, berkali-kali, berkata, berkehendak, berkeinginan, berkenaan, berlainan, berlalu, berlangsung, berlebihan, bermacam, bermacam-macam, bermaksud, bermula, bersama, bersama-sama, bersiap, bersiap-siap, bertanya, bertanya-tanya, berturut

In [20]:
#REMOVE STOPWORDS

STOPWORDS = set(stopwords.words('indonesian'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

removeoa["text_wo_stop"] = removeoa["text_wo_emoji"].apply(lambda text: remove_stopwords(text))
removeoa

,Unnamed: 0,Unnamed: 0.1,Datetime,Tweet Id,Text,Username,Text_formal,text_cleaning,Text_punct,Text_lower,text_wo_emoticons,text_wo_emoji,text_wo_stop
41,83,83,2015-07-30 19:18:48,6.268343e+17,Avanza bakal tampil dengan mesin baru yang leb...,BinandaSoraten,avanza bakal tampil dengan mesin baru yang leb...,avanza bakal tampil dengan mesin baru yang leb...,avanza bakal tampil dengan mesin baru yang leb...,avanza bakal tampil dengan mesin baru yang leb...,avanza bakal tampil dengan mesin baru yang leb...,avanza bakal tampil dengan mesin baru yang leb...,avanza tampil mesin irit
93,283,283,2015-07-30 16:40:43,6.267946e+17,Pkoknya. Nmpk jak avanza putih. Eih. Pkoknya a...,bubuibuy,pkoknya. nampak jak avanza putih. eih. pokokny...,pkoknya. nampak jak avanza putih. eih. pokokny...,pkoknya nampak jak avanza putih eih pokoknya a...,pkoknya nampak jak avanza putih eih pokoknya a...,pkoknya nampak jak avanza putih eih pokoknya a...,pkoknya nampak jak avanza putih eih pokoknya a...,pkoknya nampak jak avanza putih eih pokoknya a...
95,286,286,2015-07-30 16:37:31,6.267938e+17,Bsa plastik hihhhi versi avanza @midiahn @regg...,ladodeng,bisa plastik hihhhi versi avanza @midiahn @reg...,bisa plastik hihhhi versi avanza,bisa plastik hihhhi versi avanza,bisa plastik hihhhi versi avanza,bisa plastik hihhhi versi avanza,bisa plastik hihhhi versi avanza,plastik hihhhi versi avanza
104,301,301,2015-07-30 16:17:15,6.267887e+17,Min @torabikacoffe ada undian mobil Avanza gak...,_punyanyali,min @torabikacoffe ada undian mobil avanza eng...,min ada undian mobil avanza enggak sih.? tema...,min ada undian mobil avanza enggak sih teman ...,min ada undian mobil avanza enggak sih teman ...,min ada undian mobil avanza enggak sih teman ...,min ada undian mobil avanza enggak sih teman ...,min undian mobil avanza sih teman kupon hadiah...
116,313,313,2015-07-30 15:33:07,6.267775e+17,"Ngomong-ngomong lipstik malatuslibu, kemaren a...",falla_adinda,"ngomong-ngomong lipstik malatuslibu, kemarin a...","ngomong-ngomong lipstik malatuslibu, kemarin a...",ngomongngomong lipstik malatuslibu kemarin ada...,ngomongngomong lipstik malatuslibu kemarin ada...,ngomongngomong lipstik malatuslibu kemarin ada...,ngomongngomong lipstik malatuslibu kemarin ada...,ngomongngomong lipstik malatuslibu kemarin tem...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
54634,5454,900,2016-02-01 23:12:35,6.942973e+17,@ToyotaID saya beli inova terbaru ketika datan...,6a1275908ca145b,@toyotaid saya beli inova terbaru ketika datan...,saya beli inova terbaru ketika datang enggak ...,saya beli inova terbaru ketika datang enggak ...,saya beli inova terbaru ketika datang enggak ...,saya beli inova terbaru ketika datang enggak ...,saya beli inova terbaru ketika datang enggak ...,beli inova terbaru cek detail pas cuci mobil c...
54635,5455,901,2016-02-01 13:40:05,6.941532e+17,@Tantan837 @detikoto @ToyotaID kalau kece itu...,viantmaulana,@tantan837 @detikoto @toyotaid kalau kece itu ...,kalau kece itu mobilnya dikasih gratis baru...,kalau kece itu mobilnya dikasih gratis baru...,kalau kece itu mobilnya dikasih gratis baru...,kalau kece itu mobilnya dikasih gratis baru...,kalau kece itu mobilnya dikasih gratis baru...,kece mobilnya dikasih gratis kece hehehehe
54642,5462,908,2016-02-01 07:23:03,6.940584e+17,@toyotaid @detikoto RT @Ikmal42132: @degreetyu...,irmayuli_1,@toyotaid @detikoto rt @ikmal42132: @degreetyu...,rt pengin dong tau soal fortuner yang baru...,rt pengin dong tau soal fortuner yang baru...,rt pengin dong tau soal fortuner yang baru...,rt pengin dong tau soal fortuner yang baru...,rt pengin dong tau soal fortuner yang baru...,rt pengin tau fortuner baruminimal spesifikasi...
54649,5469,915,2016-02-01 03:49:21,6.940046e+17,Pagi2 sudah disuguhi teman2 pecinta pahlawan I...,renny_amanda,pagi2 sudah disuguhi teman-teman pecinta pahla...,pagi sudah disuguhi teman-teman pecinta pahlaw...,pagi sudah disuguhi temanteman pecinta pahlawa...,pagi sudah disuguhi temanteman pecinta pahlawa...,

#### Saving Data
Finally the preprocess step is done ! But remember that our tweets are still in Indonesian. Therefore, we need to translate it to English before we do the step of Sentiment Analysis. 

I still couldn't make the translation coding worked in Python so I decided to translate the preprocessed tweets in google spreadsheet using 'GOOGLETRANSLATE' formula.

In [21]:
#convert data into excel
removeoa.to_excel('initiation_preprocess to translate_preprocessed.xlsx')